In [1]:
import json
import pandas as pd
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
import torch.optim as optim
from tqdm import tqdm, trange
from transformers import AutoModel, AutoTokenizer
from torch.optim.lr_scheduler import MultiplicativeLR
import matplotlib.pyplot as plt
import os

## Device Setting & data directory

In [2]:
path = os.getcwd()
data_dir = os.path.dirname(path)
device = torch.device('cuda:0')
batch_size = 64

## Tokenizer

In [3]:
# Load_transformer
print('Loading transformers...')
transformer_tag = "activebus/BERT-DK_rest"
tokenizer = AutoTokenizer.from_pretrained(transformer_tag)
transformer = AutoModel.from_pretrained(transformer_tag, add_pooling_layer=False)
transformer.to(device)


Loading transformers...


Some weights of the model checkpoint at activebus/BERT-DK_rest were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

## Load Data 

In [4]:
def load_data(split_name='train'):
    print('Loading {} data...'.format(split_name))
    df = pd.read_csv(data_dir + f'/{split_name}.csv')
    return df
df = load_data('Combined_News_DJIA')
stock_df = load_data('upload_DJIA_table')
df['price'] = stock_df.Close

train_df = df[:int(len(df) * 0.9)]
valid_df = df[int(len(df) * 0.9):]
num_train_batches = (len(train_df) + batch_size - 1) // batch_size
num_valid_batches = (len(valid_df) + batch_size - 1) // batch_size
# news = pd.read_csv("../Combined_News_DJIA.csv")

print("Loaded")
# train_df

Loading Combined_News_DJIA data...
Loading upload_DJIA_table data...
Loaded


## Model 
Pre_trained_BERT Finished <br>
Need Linear & LSTM


In [5]:
class Model(nn.Module):
    def __init__(self, hidden_size, num_linear=1):
        super(Model, self).__init__()
        self.pooler = nn.Sequential(
            nn.Linear(768, 768),
            nn.Tanh(),
        )
        #self.layers = nn.ModuleList([nn.Linear(768 + 1, hidden_size)])
        self.layers = nn.ModuleList([nn.Linear(256, hidden_size)])
        self.layers.extend([nn.Linear(hidden_size, hidden_size) for i in range(num_linear)])
        self.dropout = nn.Dropout(p=0.5)
        self.output = nn.Linear(hidden_size, 1)
        
#         Add LSTM HERE
        self.LSTM = nn.LSTM(input_size=769,hidden_size=256,
                           num_layers=1,batch_first=True)
#        End LSTM HERE
    def forward(self, text_embeddings, other_inputs):
        pooled_output = self.pooler(text_embeddings)  # (batch_size, 768)
        h = torch.cat([pooled_output, other_inputs], 1)  # (batch_size, 768 + 1)
        print(h)
#         Add LSTM HERE
        #print(h[0])
        #print(h)
        h_size = list(h.shape)
        h = h.reshape(1,h_size[0],h_size[1])
        #print(h)
        l = torch.zeros([1],dtype=torch.float32)
        l = l.add(h_size[0])

        #pack_sequence(h)
        #print(h.shape)
        h = pack_padded_sequence(h,lengths=l,batch_first=True)
        #print(h.data.shape)
        #print(h)

        h, (hn, cn) = self.LSTM(h, None)
        h = h.data
#         End LSTM HERE

        for layer in self.layers:
            h = nn.functional.leaky_relu(layer(h))  # (batch_size, hidden_size)
            h = self.dropout(h)
        o = self.output(h)  # (batch_size, 1)
        print(o)
        return o

## Train hyperparameter

In [6]:
num_epochs = 10
lr = 0.001
criterion = nn.BCEWithLogitsLoss()

hidden_size = 128
num_linear = 1

records = []

In [7]:
def compute_predictions(outputs):
    logits = torch.sigmoid(outputs)
    threshold = 0.5
    predictions = torch.zeros(len(logits),1).to(device)
    for i in range(len(logits)):
        if logits[i] < threshold:
            predictions[i] = 0
        else:
            predictions[i] = 1
    return predictions

# Brute concatenate TopK News
def make_input_batch(i_batch, df, batch_size):
    rows = df[i_batch* batch_size : min((i_batch+1) * batch_size, len(df))]
    text = rows[rows.columns.difference(['price', 'Label', 'Date'])]
    text = text.apply(lambda x :' '.join(x.astype(str)),1).tolist()
    text_inputs = tokenizer(text, padding='max_length', truncation=True, return_tensors="pt")
    text_inputs = {k : v.to(device) for k, v in text_inputs.items()}
    with torch.no_grad():
        text_embeddings = transformer(**text_inputs, return_dict=True).last_hidden_state[:,0,:]
    other_inputs = torch.tensor([rows.price.tolist()], dtype=torch.float32).to(device) # (batch_size, 1)
    other_inputs = torch.transpose(other_inputs, 0, 1)
    train_labels = torch.tensor([rows.Label.tolist()],dtype=torch.float32).to(device)  
    train_labels = torch.transpose(train_labels, 0, 1) # (batch_size, 1)
    return text_embeddings, other_inputs, train_labels

## Training

In [8]:
# save path
config = f'hid-{hidden_size}-numlin-{num_linear}'
print(config)
save_path = data_dir + '/results/' + config

# model design
model = Model(hidden_size=hidden_size, num_linear=num_linear)
# if os.path.isfile(save_path + '/model.pt'):
#     model.load_state_dict(torch.load(save_path + '/model.pt'))
model.to(device)

# Learning Rate
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
lmbda = lambda epoch: 0.95
scheduler = MultiplicativeLR(optimizer, lr_lambda=lmbda)
total_acc = 0
print(total_acc)
losses, acc_train, acc_valid = [], [], []
print("Start Training")
for epoch in range(num_epochs):
    # train
    running_loss = 0.0
    total_acc = 0
    model.train()

    # shuffle train data
    train_df = train_df.sample(frac=1, random_state=epoch).reset_index(drop=True)

    for i_batch in trange(num_train_batches, desc='ep {:2d}'.format(epoch + 1)):
        text_embeddings, other_inputs, train_labels = make_input_batch(i_batch, train_df, batch_size)

        # train step
        optimizer.zero_grad()
        outputs = model(text_embeddings, other_inputs)
        
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()

        # record
        losses.append(loss.cpu().detach().item())
        running_loss += losses[-1]
        predictions = compute_predictions(outputs)
        total_acc += (predictions == train_labels).sum().item()

    print(' loss: %.6f,  train acc: %.6f' % (running_loss / len(train_df), total_acc / len(train_df)))
    acc_train.append(total_acc / len(train_df))
    scheduler.step()

    # validate
    if epoch % 1 == 0:
        model.eval()
        total_acc = 0
        ## need to find valid data
        with torch.no_grad():
            for i_batch in trange(num_valid_batches, desc='valid'):
                text_embeddings, other_inputs, valid_labels = make_input_batch(i_batch, valid_df, batch_size)
                outputs = model(text_embeddings, other_inputs)
                predictions = compute_predictions(outputs)
                total_acc += (predictions == valid_labels).sum().item()

        print('valid acc', total_acc / len(valid_df))
        acc_valid.append(total_acc / len(valid_df))

print('Finished Training')

if not os.path.isdir(save_path):
    os.mkdir(save_path)
torch.save(model.state_dict(), save_path + '/model.pt')

record = json.dumps({
    'losses' : losses, 'acc_train' : acc_train, 'acc_valid' : acc_valid, 
    'num_epochs' : num_epochs, 'batch_size' : batch_size, 'lr' : lr, 'hidden_size':hidden_size, 'num_linear':num_linear,
    'transformer_tag' : transformer_tag
}, sort_keys=True, indent=4)
records.append(record)
with open(save_path + f'/record-{num_epochs}.json', 'w') as f:
    f.write(record)

hid-128-numlin-1


ep  1:   0%|                                                                                    | 0/28 [00:00<?, ?it/s]

0
Start Training


ep  1:   4%|██▋                                                                         | 1/28 [00:02<01:14,  2.77s/it]

tensor([[-3.5706e-01, -2.2773e-01, -2.8686e-01,  ...,  1.1870e-01,
         -1.8938e-01,  1.0341e+04],
        [-3.1172e-01, -3.0280e-01, -2.5644e-01,  ...,  1.3254e-01,
         -2.7181e-01,  1.6429e+04],
        [-2.9156e-01, -3.0987e-01, -2.8559e-01,  ...,  1.1949e-01,
         -2.2765e-01,  1.5338e+04],
        ...,
        [-3.9015e-01, -2.3314e-01, -2.7727e-01,  ...,  1.5040e-01,
         -2.2969e-01,  8.8813e+03],
        [-3.8658e-01, -2.7444e-01, -3.1971e-01,  ...,  1.8218e-01,
         -1.8384e-01,  1.6513e+04],
        [-3.8697e-01, -2.5807e-01, -2.4641e-01,  ...,  8.1206e-02,
         -2.0088e-01,  1.0940e+04]], device='cuda:0', grad_fn=<CatBackward>)
tensor([[ 0.1504],
        [ 0.0576],
        [ 0.1105],
        [ 0.1299],
        [ 0.1229],
        [ 0.1286],
        [ 0.0881],
        [-0.0137],
        [ 0.1603],
        [ 0.0555],
        [ 0.1294],
        [ 0.1956],
        [ 0.1095],
        [ 0.2127],
        [ 0.0643],
        [ 0.0705],
        [ 0.1953],
     

...

tensor([[-2.6459e-01, -1.9816e-01, -2.3166e-01,  ...,  4.3366e-02,
         -2.4275e-01,  7.3657e+03],
        [-4.4351e-01, -1.6997e-01, -2.4599e-01,  ...,  5.0830e-02,
         -2.2045e-01,  7.4660e+03],
        [-3.6358e-01, -2.7964e-01, -2.1717e-01,  ...,  1.4754e-01,
         -2.0238e-01,  7.5556e+03],
        ...,
        [-3.0289e-01, -2.8608e-01, -3.0153e-01,  ...,  9.7022e-02,
         -2.0641e-01,  7.5523e+03],
        [-3.2801e-01, -2.4983e-01, -2.3538e-01,  ...,  1.0617e-01,
         -3.3416e-01,  7.9973e+03],
        [-3.5315e-01, -2.7097e-01, -2.6696e-01,  ...,  1.3142e-01,
         -2.4509e-01,  8.4248e+03]], device='cuda:0')
tensor([[0.1251],
        [0.1333],
        [0.1346],
        [0.1347],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [

valid:  75%|█████████████████████████████████████████████████████████▊                   | 3/4 [00:07<00:02,  2.60s/it]

tensor([[-4.3092e-01, -2.7025e-01, -3.0428e-01,  ...,  1.7351e-01,
         -2.1021e-01,  8.2736e+03],
        [-3.9687e-01, -1.6447e-01, -3.1085e-01,  ...,  2.0987e-01,
         -1.9773e-01,  8.4973e+03],
        [-4.0648e-01, -2.3350e-01, -3.5499e-01,  ...,  1.4228e-01,
         -2.3323e-01,  8.8352e+03],
        ...,
        [-2.5792e-01, -2.7998e-01, -2.9789e-01,  ...,  6.9041e-02,
         -1.5714e-01,  1.1430e+04],
        [-3.6105e-01, -3.1575e-01, -2.2100e-01,  ...,  1.0072e-01,
         -2.3743e-01,  1.1417e+04],
        [-3.4809e-01, -2.3759e-01, -2.0738e-01,  ...,  1.4380e-01,
         -2.2184e-01,  1.1349e+04]], device='cuda:0')
tensor([[0.1251],
        [0.1333],
        [0.1346],
        [0.1347],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [0.1348],
        [

valid: 100%|█████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.02s/it]

tensor([[-3.0299e-01, -2.0548e-01, -3.7163e-01,  ...,  2.2204e-01,
         -2.4157e-01,  1.1479e+04],
        [-5.2351e-01, -1.3666e-01, -4.0322e-01,  ...,  1.6678e-01,
         -2.3607e-01,  1.1660e+04],
        [-2.8602e-01, -2.1112e-01, -2.7816e-01,  ...,  2.0929e-01,
         -2.4698e-01,  1.1616e+04],
        ...,
        [-4.1360e-01, -3.0982e-01, -2.5829e-01,  ...,  8.7611e-02,
         -2.1257e-01,  1.1642e+04],
        [-4.2525e-01, -2.9543e-01, -3.0947e-01,  ...,  7.3900e-02,
         -2.2249e-01,  1.1782e+04],
        [-2.6573e-01, -2.9025e-01, -3.4377e-01,  ...,  1.3080e-01,
         -1.8334e-01,  1.1734e+04]], device='cuda:0')
tensor([[0.1251],
        [0.1333],
        [0.1346],
        [0.1347],
        [0.1348],
        [0.1348],
        [0.1348]], device='cuda:0')
valid acc 0.5376884422110553
Finished Training


## Record

In [9]:
records_p = data_dir + f'results/records-{num_epochs}.json'

with open(records_p, 'r') as f:
    records = json.loads(f.read())
records = [json.loads(record) for record in records]
records = {(r['hidden_size'], r['num_linear']) : r for r in records}

fig, axs = plt.subplots(len(hidden_sizes), len(num_linears), sharex=True, sharey=True, figsize=(12,8))
axs = axs if len(hidden_sizes) == 1 and len(num_linears) == 1 else axs[ihs, inl]
for ihs, hidden_size in enumerate(hidden_sizes):
    for inl, num_linear in enumerate(num_linears):
        r = records[(hidden_size, num_linear)]
        axs.plot([i for i in range(num_epochs)], r['acc_train'])
        axs.plot([i for i in range(0, num_epochs, 3)], r['acc_valid'], 'tab:orange')
        axs.set_title('hs={}, nl={} ({:.1f})'.format(hidden_size, num_linear, r['acc_valid'][-1]*100))

# for ax in axs.flat:
#     ax.set(xlabel='epochs', ylabel='accuracy')
#     # Hide x labels and tick labels for top plots and y ticks for right plots.
#     ax.label_outer()

fig.savefig(data_dir + f'results/plt-{num_epochs}.png')
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\COMP5214 WORKSPACE\\project1\\COMP5214_Tradingresults/records-10.json'